In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split
import xgboost as xgb

In [ ]:
df = pd.read_csv('./exec2/train.csv')
X = df.drop(['id', 'Premium Amount'], axis=1)
y = df['Premium Amount']

In [ ]:
# put your answer here
cat_feat = X.select_dtypes(
    include=["object"]
).columns.tolist()

num_feat = X.select_dtypes(
    include=["float64", "int64"]
).columns.tolist()
# for col in cat_feat:
#    X[col] = X[col].astype('category')

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.99, random_state=42
)
# dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
# dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
# n = 100
# model = xgb.train(
#    params=params,
#    dtrain=dtrain_reg,
#    num_boost_round=n,
# )

params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
}

# put your answer here
# Preprocessing for categorical data

# numerical_transformer = StandardScaler()


numerical_transformer = SimpleImputer(strategy='mean', missing_values=np.nan)

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_feat),
        ('cat', categorical_transformer, cat_feat)
    ]
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ("regressor", GradientBoostingRegressor(**params)),
])
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)

